<a href="https://colab.research.google.com/github/CY-Chang-tw/Information-Retrieval-using-Pyserini/blob/main/IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 38.4 MB/s eta 0:00:00


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [4]:
from google.colab import drive
import pandas as pd

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
data = pd.read_json("input/wiki_2021_08_05_1215639.json")

In [9]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215639 entries, 0 to 1215638
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   id        1215639 non-null  int64 
 1   title     1215639 non-null  object
 2   articles  1215639 non-null  object
dtypes: int64(1), object(2)
memory usage: 27.8+ MB


,id,title,articles
0,1,福斯家庭電影頻道,福斯家庭電影頻道（）是一個播放電影的亞洲電視頻道，由福斯國際頻道持有與出品，STAR衛視發佈...
1,2,六羰基鎢,六羰基鎢是一種配位化合物，化學式爲W(CO)。經過它得到了第一個雙氫配合物。它是無色固體，和...
2,3,吉米·史奴卡,吉米·史奴卡（，），本名詹姆斯·威廉·瑞荷（），是世界摔角娛樂（WWE）旗下退休職業摔角選手...
3,4,達文波特-欣策爾序列,在組合數學中，達文波特–欣策爾序列是指對任意兩個符號交替出現的次數作出限制的序列。達文波特–...
4,5,米基爾·安祖·莫亞,米基爾‧安祖‧莫亞（，)簡稱 莫亞 。西班牙足球運動員，司職門將。莫亞生於西班牙馬約卡島上的...


In [10]:
!python -m pyserini.index.lucene \
  --collection CoreCollection \
  --input input \
  --fields "title" "articles" \
  --language zh \
  --bm25.accurate \
  --index indexes/wiki_2021_08_05_1215639 \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw 

2023-03-16 04:36:33,824 INFO  [main] index.IndexCollection (IndexCollection.java:391) - Setting log level to INFO
2023-03-16 04:36:33,828 INFO  [main] index.IndexCollection (IndexCollection.java:394) - Starting indexer...
2023-03-16 04:36:33,828 INFO  [main] index.IndexCollection (IndexCollection.java:395) - ============ Loading Parameters ============
2023-03-16 04:36:33,828 INFO  [main] index.IndexCollection (IndexCollection.java:396) - DocumentCollection path: input
2023-03-16 04:36:33,829 INFO  [main] index.IndexCollection (IndexCollection.java:397) - CollectionClass: CoreCollection
2023-03-16 04:36:33,830 INFO  [main] index.IndexCollection (IndexCollection.java:398) - Generator: DefaultLuceneDocumentGenerator
2023-03-16 04:36:33,831 INFO  [main] index.IndexCollection (IndexCollection.java:399) - Threads: 1
2023-03-16 04:36:33,832 INFO  [main] index.IndexCollection (IndexCollection.java:400) - Language: zh
2023-03-16 04:36:33,833 INFO  [main] index.IndexCollection (IndexCollection.

In [11]:
import time
from pyserini.search.lucene import LuceneSearcher
from pyserini.index.lucene import IndexReader
import itertools

searcher = LuceneSearcher('indexes/wiki_2021_08_05_1215639')
time_start = time.time() #開始計時
hits = searcher.search('家庭')
time_end = time.time()    #結束計時

time_inverted_index = time_end - time_start   #執行所花時間
print('time cost', time_inverted_index, 's')

print('hits: ')
for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
    print(hits[i].raw)
print()

# Initialize from an index path:
index_reader = IndexReader('indexes/wiki_2021_08_05_1215639')

# access basic index statistics
print('basic index statistics: ')
index_reader.stats()

term = '家庭'


# Analyze the term.
analyzed = index_reader.analyze(term)
print(f'The analyzed form of "{term}" is "{analyzed[0]}"')
print()

# Fetch and traverse postings for an analyzed term:
postings_list = index_reader.get_postings_list(analyzed[0], analyzer=None)
print('Posting: ')
for posting in postings_list:
    print(f'docid={posting.docid}, tf={posting.tf}, pos={posting.positions}')
print()


time cost 0.1923537254333496 s
hits: 
 1      7.93350
庭 (機關) 
 2      7.16420
家 
 3      6.79880
家 (小說) 
 4      6.46890
夢·想·家 
 5      6.46890
家，N次方 
 6      6.46890
家 (消歧義) 
 7      6.16950
家·多點嘗試 
 8      5.89660
家 (公視電視劇) 
 9      5.89660
家 (陳潔儀歌曲) 
10      5.89660
家 (2007年電視劇) 

basic index statistics: 
The analyzed form of "家庭" is "家"

Posting: 
docid=203315, tf=1, pos=[0]
docid=367764, tf=1, pos=[0]
docid=388074, tf=1, pos=[0]
docid=470856, tf=1, pos=[0]
docid=620346, tf=1, pos=[0]
docid=624586, tf=1, pos=[0]
docid=639641, tf=1, pos=[0]
docid=725540, tf=1, pos=[2]
docid=810612, tf=1, pos=[0]
docid=1128238, tf=1, pos=[0]

